## Predictive modeling - Linear Regression model (Daily mean tempreture)

In [1]:
import pandas as pd
import numpy as np

In [2]:
kandy_weather = pd.read_csv('../data/processed/kandy_weather_cleaned.csv', index_col=0, parse_dates=['time'])

In [3]:
#drop highly correlated columns 
kandy_weather = kandy_weather.drop(columns=['max_temperature', 'min_temperature', 'apparent_temperature_mean'])

In [ ]:
# Define features and target variables
variables = ['weather_code', 'shortwave_radiation', 'rainfall', 'max_wind_speed', 'max_wind_gust', 'dominant_Wind_Direction', 'evapotranspiration', 'mean_temperature', 'rain_tomorrow']
target = 'mean_temperature'

x = kandy_weather.drop(columns=[target])
y = kandy_weather[target]

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)




